In [1]:
import setup
setup.init()

In [2]:
from mypermit import permit_client


permit = permit_client

In [3]:
await permit.api.users.list()

PaginatedResultUserRead(data=[UserRead(key='1', id=UUID('6b40ddfa-306b-4f68-9d83-c1cdf3ebaf1c'), organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935'), project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6'), environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf'), associated_tenants=[UserInTenant(tenant='default', roles=['admin'], status=<UserStatus.pending: 'pending'>, resource_instance_roles=[])], roles=[UserRole(role='admin', tenant='default')], created_at=datetime.datetime(2025, 7, 22, 5, 26, 41, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 22, 5, 30, 43, tzinfo=datetime.timezone.utc), email=None, first_name='soumo', last_name=None, attributes=None), UserRead(key='2', id=UUID('aaed34be-2e28-425f-b281-f6c03334e60b'), organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935'), project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6'), environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf'), associated_tenants=[], roles=[], created_at=datetime.d

In [4]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [5]:
for u in User.objects.all():
    user_data = {
        "key": f"{u.id}",
        "first_name": f"{u.username}",
    }
    response = await permit.api.users.sync(user_data)
    print(response)



key='1' id=UUID('6b40ddfa-306b-4f68-9d83-c1cdf3ebaf1c') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') associated_tenants=[UserInTenant(tenant='default', roles=['admin'], status=<UserStatus.pending: 'pending'>, resource_instance_roles=[])] roles=[UserRole(role='admin', tenant='default')] created_at=datetime.datetime(2025, 7, 22, 5, 26, 41, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 22, 5, 30, 43, tzinfo=datetime.timezone.utc) email=None first_name='soumo' last_name=None attributes=None
key='2' id=UUID('aaed34be-2e28-425f-b281-f6c03334e60b') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') associated_tenants=[] roles=[] created_at=datetime.datetime(2025, 7, 22, 5, 26, 41, tzinfo=datetime.timezone.utc) updated

In [6]:
await permit.check("1", "read", "document")

True

In [7]:
resources = [
    {
        "name": "Documents",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [8]:
from permit import PermitApiError

async def get_or_create_resource(resource_data:dict):
    resource_key = resource_data.get('key')
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            raise e
    return _resource

In [9]:
for resource_data in resources:
    response = await get_or_create_resource(resource_data)
    print(response)

key='document' id=UUID('fb6d4cab-aa72-4131-a641-bff349ac1944') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') created_at=datetime.datetime(2025, 7, 22, 6, 44, 52, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 22, 6, 44, 52, tzinfo=datetime.timezone.utc) name='Documents' urn='prn:soumo-789:default:document' description=None actions={'read': ActionBlockRead(name='read', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('6444e189-1f74-4e9b-88ee-f6ddfc11b470'), key='read'), 'create': ActionBlockRead(name='create', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('529e2530-79ea-42d9-8ae8-85aeb9741a97'), key='create'), 'update': ActionBlockRead(name='update', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('35ef32ba-b242-4039-a0f4-49ee1994cb3

In [10]:
# read_only_permissions = [
#     "document:read",
#     "movie_disovery:search",
#     "movie_disovery:detail",
# ]


In [11]:
roles= [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": {
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail",
        }
    },
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions":  [
            "document:read",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }
]

In [12]:
from permit import PermitApiError

async def get_or_create_role(role_data:dict):
    _key = role_data.get('key')
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            raise e
    return _resource

In [13]:
for role_data in roles:
    response = await get_or_create_role(role_data)
    print(response)

name='Manager' description=None permissions=['document:create', 'movie_disovery:detail', 'movie_disovery:search', 'document:read', 'document:update'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='manager' id=UUID('0906b770-1c8b-453b-8904-4bb4b7242108') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') created_at=datetime.datetime(2025, 7, 22, 6, 48, 48, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 7, 22, 6, 48, 48, tzinfo=datetime.timezone.utc)
name='Viewer' description=None permissions=['document:read', 'movie_disovery:search', 'movie_disovery:detail'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='viewer' id=UUID('7cc65f0f-f5cc-418d-9252-8d90b7c9f7f4') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-445

In [14]:
# await permit.api.roles.delete("viewer")

In [15]:

user = User.objects.all().first()

In [16]:
user

<User: soumo>

In [17]:
user.id

1

In [18]:
response = await permit.api.users.assign_role({
    "user": f"{user.id}",
    "role": "admin",
    "tenant": "default"
})
print(response)

id=UUID('bd40fea9-1c37-429a-a269-6a57d02cedfa') user='1' role='admin' tenant='default' resource_instance=None resource_instance_id=None user_id=UUID('6b40ddfa-306b-4f68-9d83-c1cdf3ebaf1c') role_id=UUID('95c8b5ca-34ff-43bb-a9ee-4d0fb002b709') tenant_id=UUID('52af0810-7663-4b92-be3e-092126518117') organization_id=UUID('c148024e-3c94-40a0-b3a7-c2ed580c1935') project_id=UUID('0651285d-07cb-4459-9a3b-a7d77da6c2b6') environment_id=UUID('4783bfa6-5e6c-427d-9118-646f2eb01edf') created_at=datetime.datetime(2025, 7, 22, 6, 56, 24, tzinfo=datetime.timezone.utc)


In [19]:
await permit.check(f"{user.id}", "read", "document")

True